In [1]:
import math
import time
import pandas as pd
import numpy as np
import numpy.linalg as la

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor

In [2]:
def load_data(path):
    df = pd.read_csv(path)
    data = df.drop(columns=['Unnamed: 0'])
    
    return data

In [3]:
def preprocess_data(data, time_len, rate, seq_len, pre_len):
    train_size = int(time_len * rate)
    train_data = data[0:train_size]
    test_data = data[train_size:time_len]
    
    trainX, trainY, testX, testY = [], [], [], []
    for i in range(len(train_data) - seq_len - pre_len):
        a = train_data[i: i + seq_len + pre_len]
        trainX.append(a[0 : seq_len])
        trainY.append(a[seq_len : seq_len + pre_len])
    for i in range(len(test_data) - seq_len -pre_len):
        b = test_data[i: i + seq_len + pre_len]
        testX.append(b[0 : seq_len])
        testY.append(b[seq_len : seq_len + pre_len])
        
    return trainX, trainY, testX, testY

In [4]:
def getTestY(data):
    tmp_scaler = StandardScaler()
    tmp_data = tmp_scaler.fit_transform(data)
    tmp_data = tmp_scaler.inverse_transform(tmp_data)
    _, _, _, testY = preprocess_data(tmp_data, data.shape[0], train_rate, seq_len, pre_len)
    testY = np.array(testY)
    testY = np.reshape(testY, [-1, pre_len])
    
    return testY

In [5]:
 def evaluation(a,b):
    rmse = math.sqrt(mean_squared_error(a,b))
    mae = mean_absolute_error(a, b)
    mape = mean_absolute_percentage_error(a, b)
    F_norm = la.norm(a-b)/la.norm(a)
    
    return rmse, mae, mape, 1-F_norm

In [6]:
def predictSVR(data, testY):
    scaler = StandardScaler()
    data = scaler.fit_transform(data)
    
    rmses, maes, mapes, accs = [], [], [], []
    for i in range(data.shape[1]):
        print('Node', i)
        start = time.time()
        a = data[:, i]
        aX, aY, tX, tY = preprocess_data(a, data.shape[0], train_rate, seq_len, pre_len)
        
        aX = np.array(aX)
        aX = np.reshape(aX, [-1, seq_len])
        aY = np.array(aY)
        aY = np.reshape(aY, [-1, pre_len])

        tX = np.array(tX)
        tX = np.reshape(tX, [-1, seq_len])
        tY = np.array(tY)
        tY = np.reshape(tY, [-1, pre_len])
        
        print('(', aX.shape, aY.shape, tX.shape, tY.shape, round(aX.mean(),3), ')')
        reg = MultiOutputRegressor(SVR(kernel='linear'))
        reg.fit(aX, aY)
        pred = reg.predict(tX)
        
        mean = scaler.mean_[i]
        std = np.sqrt(scaler.var_[i])
        pred = pred*std + mean
        tY = tY*std + mean
        if i==data.shape[1]-1: tY = testY[tY.shape[0]*i:]
        else: tY = testY[tY.shape[0] * i:tY.shape[0] * (i+1)]
        
        rmse, mae, mape, acc = evaluation(tY, pred)
        rmses.append(rmse)
        maes.append(mae)
        mapes.append(mape)
        accs.append(acc)
        
        print('(', rmse, mae, mape, acc, time.time() - start)
    
    print('RMSE: ' + str(np.mean(rmses)) + ', MAE: ' + str(np.mean(maes)) + ', MAPE: ' + str(np.mean(mapes)) + ', ACC: ' + str(np.mean(accs)))

In [7]:
train_rate = 0.8
seq_len = 12
pre_len = 6

In [8]:
path_0 = '../Data/Gangnam/speed_gangnam_0.csv'
path_5 = '../Data/Gangnam/speed_gangnam_5.csv'
path_10 = '../Data/Gangnam/speed_gangnam_10.csv'
path_20 = '../Data/Gangnam/speed_gangnam_20.csv'

In [9]:
masterData = load_data(path_0)
print(masterData.shape, (masterData == 0).sum().sum())
testY = getTestY(masterData)
testY.shape, (testY == 0).sum().sum()

(2880, 506) 0


((282348, 6), 0)

### Unobserved Nodes = 0%

In [10]:
data = load_data(path_0)
print(data.shape, (data == 0).sum().sum())
predictSVR(data, testY)

(2880, 506) 0
Node 0
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.004 )
( 10.751276746671783 7.618165517823226 0.24987702501566178 0.663807065587046 2.580106019973755
Node 1
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.032 )
( 13.513944003597526 9.983707317518416 0.31859579526579085 0.5819834893230527 2.0565028190612793
Node 2
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.095 )
( 12.38422530722103 9.593065952740226 0.3445397670813946 0.6169693301328221 2.2809057235717773
Node 3
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.001 )
( 14.85566191070618 12.704713762891402 0.5056856811059621 0.5432540322681432 2.537245750427246
Node 4
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.006 )
( 16.35642354792104 12.753406233065858 0.37963257523328253 0.4982359402774631 2.4733614921569824
Node 5
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.02 )
( 13.517687473790497 9.821299680585442 0.283619375793492 0.5873567308754998 2.9770724773406982
Node 6
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.012 )
( 12.044179

( 19.224733525735886 16.193852560915616 0.46838074205210445 0.42200906352638523 3.07877254486084
Node 54
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.009 )
( 17.580771171540245 14.218688810381247 0.404709172094301 0.4702620663437438 3.4447927474975586
Node 55
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.008 )
( 10.672558429607278 7.685206048928667 0.25891402037899675 0.6774547621202135 4.110016107559204
Node 56
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.003 )
( 13.276367038547695 10.672128125611598 0.4158717849409741 0.5998776550514461 3.322124719619751
Node 57
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.007 )
( 11.798079923696465 8.711305523855204 0.2985836650600638 0.6430399222017549 3.3141398429870605
Node 58
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.037 )
( 15.657903699017638 13.260032052494957 0.5291498995634356 0.5262812602865952 2.949117422103882
Node 59
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.0 )
( 12.072220544575377 9.347112127438285 0.35534926687828944 0.634386297200

( 13.283544476440028 11.653827076077137 0.594041579993633 0.5114027553250869 3.4188625812530518
Node 108
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.017 )
( 9.99139346811922 7.2048531913204235 0.2961696398499815 0.6319999432272565 3.9883406162261963
Node 109
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.012 )
( 10.706222215068735 8.071041860822115 0.3492361893653661 0.6019620557067267 3.3869478702545166
Node 110
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.033 )
( 10.261583427532901 7.943644070965159 0.3886867646946514 0.6187408968646124 3.773913860321045
Node 111
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.017 )
( 9.492549096663089 6.896255775348847 0.2984211081726486 0.6455975200527274 3.1455929279327393
Node 112
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.025 )
( 9.92946500131185 7.4951421843513755 0.3557163338006027 0.6281201142647133 3.218400001525879
Node 113
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.002 )
( 10.747176905386894 8.218409776073551 0.35949959714619445 0.596937550708

( 9.883534816474414 7.301124282550453 0.3514414631880145 0.6254227786052515 3.6856577396392822
Node 161
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.043 )
( 9.924300320667083 7.351731176131989 0.36215286760638854 0.622107810683389 3.3161814212799072
Node 162
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.005 )
( 11.90761533318878 8.762994088635523 0.3614930986150619 0.5475143475179154 2.749607563018799
Node 163
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.026 )
( 10.833004410072258 8.015873953512466 0.42135351452099806 0.5846740359708482 2.9102210998535156
Node 164
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.003 )
( 12.92759926643246 9.983088562613954 0.5510431092401317 0.5060502585454083 3.8866121768951416
Node 165
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.005 )
( 11.961652687593192 9.466738669608318 0.5033833959964632 0.5460082592142459 2.974053144454956
Node 166
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.062 )
( 18.110345305770686 15.059398193477515 0.8145922253752537 0.30420427468

( 17.24061436711218 15.489606823231009 0.7303456442526214 0.40389429881481687 3.688141107559204
Node 214
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.039 )
( 15.53408179021132 12.37269937665637 0.4080143039968498 0.46449116395972656 3.674180269241333
Node 215
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.083 )
( 12.403852816055121 8.970628116675789 0.29846126592740757 0.5746172897241855 2.785555839538574
Node 216
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.064 )
( 10.199590483962494 7.229311959263673 0.28649211714960227 0.6489019133760987 3.5574936866760254
Node 217
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.016 )
( 17.84729291550975 16.136382902840733 0.7446796498742864 0.38836650296809927 3.5295655727386475
Node 218
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.051 )
( 11.880263244475676 8.5942740277835 0.3153856618567919 0.5915242346836025 3.224384069442749
Node 219
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.023 )
( 12.200354938055053 9.8624972400882 0.44494502145482445 0.5797262039036

( 17.190489749386558 13.58212223957093 0.38568510911202125 0.48473162917366686 3.599381685256958
Node 267
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.003 )
( 14.564977372247704 10.797416139791531 0.33036964543457964 0.5663483699775318 3.2084248065948486
Node 268
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.029 )
( 13.427988756102465 9.750550037195191 0.28508313838453514 0.600126598692607 3.1117191314697266
Node 269
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.007 )
( 12.809498715136202 9.361207351373695 0.2943798154217813 0.6199997401542743 4.067093372344971
Node 270
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.034 )
( 20.696279551957343 17.3148603922907 0.4983605772799744 0.38877188153410824 2.8873302936553955
Node 271
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.002 )
( 13.184930580233372 9.757814364348272 0.31518839211560096 0.6105936058937191 3.421809196472168
Node 272
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.005 )
( 11.853293524957694 8.551221685407336 0.2742988505192971 0.6526

( 17.148437588674422 13.528407085671121 0.39361717718566896 0.4871859990883015 3.0469000339508057
Node 320
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.094 )
( 15.255659390799563 11.784525527591237 0.3504560190615129 0.5405704809697072 3.0188889503479004
Node 321
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.05 )
( 12.338944357642912 9.077828235587715 0.2882504987789956 0.6273710763172526 3.3570284843444824
Node 322
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.007 )
( 13.342024495585237 10.085997162041435 0.3348602399018219 0.5987518856074074 2.6189990043640137
Node 323
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.009 )
( 15.084270468459147 11.539745903051555 0.34746405577031475 0.5431465810267555 2.877345561981201
Node 324
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.02 )
( 12.398067312887093 9.540318799867656 0.3486840978555244 0.624486622199601 3.0967187881469727
Node 325
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.032 )
( 12.108323120024679 9.531334060552753 0.3862752308133519 0.63375622

( 19.390072727236465 17.68612134250151 0.9103643667920825 0.278108779986322 3.5415775775909424
Node 373
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.038 )
( 10.854517918859294 7.821393138721706 0.3083328403436025 0.5955791483462385 2.9670262336730957
Node 374
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.009 )
( 15.303838143059853 13.197677298615346 0.680140671809545 0.43368386464977915 3.5116169452667236
Node 375
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.014 )
( 38.48793403848275 36.46819994003321 1.8136134595780555 -0.43639309915601365 2.6259806156158447
Node 376
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.0 )
( 58.4075091347586 57.53765169149661 2.8081380549316983 -1.1775971009582227 2.8883285522460938
Node 377
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.034 )
( 42.50893653366263 38.99788471576022 1.9553018739915506 -0.5990556651701748 1.8869524002075195
Node 378
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.002 )
( 33.82200281580312 30.63425569880535 1.591715824819083 -0.27748154

( 12.594374428788022 10.264664350912993 0.6074767137393898 0.508635383352171 3.2772395610809326
Node 426
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.026 )
( 22.874323707516673 20.911392105985616 1.2082129020937067 0.11724961906632814 2.912217855453491
Node 427
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.071 )
( 27.96400311887078 25.42354347260175 1.4287542829834292 -0.08104355835780708 2.2908763885498047
Node 428
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.014 )
( 32.69881540709999 30.47753562370126 1.6854063598181133 -0.2675407708519848 2.935155153274536
Node 429
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.011 )
( 34.75687346447151 32.01810397580345 1.8150134950473742 -0.35847504399874763 2.0096304416656494
Node 430
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.027 )
( 37.99750365763068 36.60536632044771 2.071945369934116 -0.48380489670088656 3.045301914215088
Node 431
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.003 )
( 28.02860502015368 26.321940953030964 1.5366179891106864 -0.103198

( 12.482070648769628 9.11899791299838 0.30772766233241294 0.5798598072184247 3.235394239425659
Node 479
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.021 )
( 10.488039886547861 7.726578026196555 0.3045358853522024 0.6439350279855255 3.0627715587615967
Node 480
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.059 )
( 14.669239285823902 11.168185600332599 0.37602722992124143 0.5040466997865423 2.5651473999023438
Node 481
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.098 )
( 10.20110681079294 7.088762332321369 0.25148294570496194 0.6562087028568104 3.3001785278320312
Node 482
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.084 )
( 16.176084868232632 13.028220848105159 0.422920308668348 0.45280471414786283 3.104701280593872
Node 483
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.048 )
( 10.718991581129162 7.690814367708708 0.2690322468427812 0.6364658551232378 3.0274429321289062
Node 484
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.011 )
( 10.95408299279555 7.996049578922491 0.29230391204375655 0.63021671

### Unobserved Nodes = 5%

In [11]:
data = load_data(path_5)
print(data.shape, (data == 0).sum().sum())
predictSVR(data, testY)

(2880, 506) 72000
Node 0
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.004 )
( 10.751276746671783 7.618165517823226 0.24987702501566178 0.663807065587046 3.2572929859161377
Node 1
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.032 )
( 13.513944003597526 9.983707317518416 0.31859579526579085 0.5819834893230527 2.4714317321777344
Node 2
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.095 )
( 12.38422530722103 9.593065952740226 0.3445397670813946 0.6169693301328221 2.668398141860962
Node 3
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.001 )
( 14.85566191070618 12.704713762891402 0.5056856811059621 0.5432540322681432 2.7922072410583496
Node 4
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.006 )
( 16.35642354792104 12.753406233065858 0.37963257523328253 0.4982359402774631 2.969106912612915
Node 5
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.02 )
( 13.517687473790497 9.821299680585442 0.283619375793492 0.5873567308754998 3.1964147090911865
Node 6
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.012 )
( 12.04

( 17.580771171540245 14.218688810381247 0.404709172094301 0.4702620663437438 2.778542995452881
Node 55
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.008 )
( 10.672558429607278 7.685206048928667 0.25891402037899675 0.6774547621202135 3.4488136768341064
Node 56
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.003 )
( 13.276367038547695 10.672128125611598 0.4158717849409741 0.5998776550514461 2.773554801940918
Node 57
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.007 )
( 11.798079923696465 8.711305523855204 0.2985836650600638 0.6430399222017549 3.003974676132202
Node 58
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.037 )
( 15.657903699017638 13.260032052494957 0.5291498995634356 0.5262812602865952 2.48036789894104
Node 59
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.0 )
( 12.072220544575377 9.347112127438285 0.35534926687828944 0.6343862972002026 2.8583927154541016
Node 60
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.052 )
( 11.377578280935872 8.806733806196677 0.34096874755584405 0.6550099500995

( 10.706222215068735 8.071041860822115 0.3492361893653661 0.6019620557067267 2.9759914875030518
Node 110
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.033 )
( 10.261583427532901 7.943644070965159 0.3886867646946514 0.6187408968646124 3.415900945663452
Node 111
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.017 )
( 9.492549096663089 6.896255775348847 0.2984211081726486 0.6455975200527274 2.998955488204956
Node 112
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.025 )
( 9.92946500131185 7.4951421843513755 0.3557163338006027 0.6281201142647133 2.8623979091644287
Node 113
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.002 )
( 10.747176905386894 8.218409776073551 0.35949959714619445 0.5969375507089676 2.6478755474090576
Node 114
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.039 )
( 9.150800318586997 6.685187385645096 0.32952389502370927 0.6552831333878761 3.644259452819824
Node 115
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.016 )
( 10.214708670967186 7.709952172150579 0.36667684106611514 0.615547403947

( 10.833004410072258 8.015873953512466 0.42135351452099806 0.5846740359708482 2.6017189025878906
Node 164
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.003 )
( 12.92759926643246 9.983088562613954 0.5510431092401317 0.5060502585454083 3.340043306350708
Node 165
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.005 )
( 11.961652687593192 9.466738669608318 0.5033833959964632 0.5460082592142459 2.742671251296997
Node 166
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.062 )
( 18.110345305770686 15.059398193477515 0.8145922253752537 0.3042042746844028 2.1941444873809814
Node 167
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.058 )
( 17.70351760368119 14.324514985296739 0.7764914089515006 0.32686128802298864 2.2729575634002686
Node 168
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.005 )
( 28.91913418078668 27.229710352841355 1.4350133034419372 -0.09488405055187643 3.011918544769287
Node 169
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.008 )
( 16.204665893798605 13.352616217087585 0.6583053386050247 0.383197

( 17.84729291550975 16.136382902840733 0.7446796498742864 0.38836650296809927 3.0577855110168457
Node 218
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.051 )
( 11.880263244475676 8.5942740277835 0.3153856618567919 0.5915242346836025 2.8773086071014404
Node 219
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.023 )
( 12.200354938055053 9.8624972400882 0.44494502145482445 0.5797262039036025 2.826479911804199
Node 220
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.007 )
( 12.851775799267365 10.236020191762366 0.4568799523396801 0.5616372426507622 2.679805278778076
Node 221
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.015 )
( 10.43998150598834 8.056413860627897 0.3574390643841289 0.6430666119003161 3.01796555519104
Node 222
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.016 )
( 10.13989663498449 7.176995783297609 0.27518174322497346 0.6525130459751756 3.059823751449585
Node 223
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.024 )
( 10.791397954325854 7.950700488530987 0.31437194191997875 0.63099573129660

( 20.696279551957343 17.3148603922907 0.4983605772799744 0.38877188153410824 2.7287399768829346
Node 271
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.002 )
( 13.184930580233372 9.757814364348272 0.31518839211560096 0.6105936058937191 3.135586738586426
Node 272
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.005 )
( 11.853293524957694 8.551221685407336 0.2742988505192971 0.6526278193183797 3.4009506702423096
Node 273
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.0 )
( 33.94677599490162 32.07290621266427 1.0 0.0 0.007937908172607422
Node 274
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.0 )
( 16.35655004335226 12.72689313545631 0.36394187855574206 0.5176356180885207 2.759624481201172
Node 275
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.024 )
( 16.108961823273464 12.391186395644917 0.35000840129328087 0.528820180778318 2.8025529384613037
Node 276
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.033 )
( 17.239441440580595 13.405413205266477 0.3857075523927955 0.49234589818263563 2.6897690296173096
N

( 12.398067312887093 9.540318799867656 0.3486840978555244 0.624486622199601 3.0059316158294678
Node 325
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.032 )
( 12.108323120024679 9.531334060552753 0.3862752308133519 0.6337562294117143 3.005995750427246
Node 326
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.026 )
( 11.231435094919885 8.051425429774486 0.2863984772755086 0.6596587041761024 2.6987578868865967
Node 327
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.07 )
( 16.872563504057336 14.03715775661768 0.5952130976210849 0.49088559018945543 3.51859712600708
Node 328
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.0 )
( 33.06007268419278 31.144059139784943 1.0 0.0 0.009009599685668945
Node 329
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.044 )
( 12.491642523385071 9.532862579086057 0.3465573169161802 0.6191118157223707 3.366001605987549
Node 330
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.044 )
( 11.712644112635155 8.80341222156195 0.33194605805357624 0.642366921524148 3.3350541591644287
Node 331
(

( 33.82200281580312 30.63425569880535 1.591715824819083 -0.27748154018578886 1.9039092063903809
Node 379
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.003 )
( 11.093838470387121 8.409077000041759 0.41563376582953754 0.582300530308246 3.0199310779571533
Node 380
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.051 )
( 12.433621734190206 10.227704052004194 0.5504136628989219 0.5280749151451359 2.8842902183532715
Node 381
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.034 )
( 11.338542436270792 8.628112059518106 0.4211809988765771 0.5708648590290175 2.856309652328491
Node 382
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.0 )
( 26.381887636800453 24.49741935483871 1.0 0.0 0.008976936340332031
Node 383
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.031 )
( 10.397014555992682 7.7805633570986545 0.392899632413981 0.6044100658770073 2.8723294734954834
Node 384
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.054 )
( 10.511801377188188 7.775533473594186 0.38251714531659714 0.5979948277658371 2.5780794620513916
No

( 12.04035369771761 8.569548647653143 0.3685616227623269 0.5208616208474017 2.700777769088745
Node 433
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.01 )
( 14.464782506990344 11.288255522001753 0.44945699176135223 0.4254481838157329 2.377664804458618
Node 434
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.0 )
( 25.44722644516477 23.616129032258062 1.0 0.0 0.007960319519042969
Node 435
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.07 )
( 10.02150953128456 7.261870254332165 0.35455316917315893 0.6029947460558182 2.4265167713165283
Node 436
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.012 )
( 11.400632042776204 8.194941041894513 0.34778167850949443 0.5494699258412015 2.229043483734131
Node 437
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.018 )
( 10.969614829874342 8.218725656615245 0.43477927743760897 0.5662541100813558 2.059459924697876
Node 438
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.02 )
( 10.169182928590963 7.876543714937579 0.45430067945155234 0.5966422431292213 2.966073989868164
Node 

( 14.47868651087663 12.740826831708295 0.5840056165302691 0.5099841370836911 2.9072272777557373
Node 488
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.015 )
( 10.610465718341011 8.358001034317175 0.36758278082337154 0.638341097342287 3.1954874992370605
Node 489
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.028 )
( 17.519426010639965 15.784728028642853 0.7323977334402373 0.404560899117972 2.7246859073638916
Node 490
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.026 )
( 15.060384353980721 13.253489222419821 0.6244468084277773 0.485193218923311 3.6323232650756836
Node 491
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.025 )
( 10.39356296856954 7.4555157020007075 0.28688665511084777 0.6451050977240865 2.704773187637329
Node 492
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.012 )
( 12.305621506991612 8.987747373434722 0.310938315296078 0.5792731214416814 2.9939651489257812
Node 493
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.017 )
( 10.407343672306402 8.007131584359755 0.3387505651896047 0.6468635